## Kindle Review Sentiment Analysis
#### About Dataset
**Context**

This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

**Content**

5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

* asin - ID of the product, like B000FA64PK
-helpful - helpfulness rating of the review - example: 2/3.
-overall - rating of the product.
-reviewText - text of the review (heading).
-reviewTime - time of the review (raw).
-reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
-reviewerName - name of the reviewer.
-summary - summary of the review (description).
-unixReviewTime - unix timestamp.

**Acknowledgements**
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

**Inspiration**

-Sentiment analysis on reviews.
-Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
-Fake reviews/ outliers.
-Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
-Any other interesting analysis

## Best Practices
1. PreProcessing and Cleaning
2. Train Test Split
3. Apply BOW,TFIDF,Word2Vev
4. Train ML algorithms

In [1]:
## Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## Load dataset
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/all_kindle_review .csv')

In [3]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
data=data[['reviewText','rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
data.shape

(12000, 2)

In [6]:
## Missing Values
data.isnull().sum()

,0
reviewText,0
rating,0


In [7]:
## Unique Ratings
data['rating'].unique()

array([3, 5, 4, 2, 1])

In [8]:
## Data ratings value_count
data['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


### Pre-Processing and Cleaning

We are going to define the review as positive or negative based on ratings.
* less than 3 -> negative review
* more than 3 -> positive review

In [9]:
# positive review is 1 and negative review is 0
data['rating']=data['rating'].apply(lambda x:0 if x<3 else 1)

In [10]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [11]:
data['rating'].unique()

array([1, 0])

In [12]:
data['rating'].value_counts()

,count
rating,
1,8000
0,4000


In [13]:
# 1. Lower all the cases
data['reviewText']=data['reviewText'].str.lower()

In [14]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [15]:
!pip install BeautifulSoup4

In [16]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords

In [18]:
# 2. Removing special characters
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
# 1. Remove special characters
data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9]+','',x))
# 3. Remove the stopwords
data['reviewText']=data['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
# 4. remove URL
data['reviewText']=data['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))
# 5. Remove HTML Tags
data['reviewText']=data['reviewText'].apply(lambda x:BeautifulSoup(x,'lxml').get_text())
# 6. Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x:" ".join(x.split()))

In [19]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [20]:
## Applyong Lemmatiztion
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
def lemmatize_words(text):
  return " ".join([lm.lemmatize(word) for word in text.split()])

In [22]:
data['reviewText']=data['reviewText'].apply(lambda x:lemmatize_words(x))

In [23]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


### Train Test Split

In [24]:
##Independent varibale
X=data['reviewText']
## Dependent variable
y=data['rating']

In [25]:
X.head()

,reviewText
0,jace rankin may short he nothing mess man haul...
1,great short read didnt want put read one sitti...
2,ill start saying first four book wasnt expecti...
3,aggie angela lansbury carry pocketbook instead...
4,expect type book library pleased find price right


In [26]:
y.head()

,rating
0,1
1,1
2,1
3,1
4,1


In [27]:
X.shape,y.shape

((12000,), (12000,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
X_train

,reviewText
9182,looking forward book came double space every p...
11091,already owned book spouse forgot already part ...
6428,cool forgot request rate came make mine unreli...
288,short short story basically scene party one ni...
2626,secret service agent secrests even longer serv...
...,...
11964,downloaded book reading review usually reading...
5191,far one hottest book ive ever gotten hand ondo...
5390,even though book free reservation based majori...
860,little mushy 34must take care woman folk34 cha...


In [30]:
X_test

,reviewText
1935,really great read wish would hope find author
6494,nope tried cant read take greatest delight del...
1720,story line drug like book much mystery fan wou...
9120,read several angel book one work didnt really ...
360,possibly worst book ever read beginning positi...
...,...
1195,enjoyed read think fan humorous must err use l...
11877,pleasantly surprised book enjoyed m dubois tol...
5421,love best friend since 15 year old 30 he servi...
3855,fascinating book enough twist turn keep readin...


In [31]:
y_train

,rating
9182,1
11091,0
6428,1
288,0
2626,1
...,...
11964,0
5191,1
5390,0
860,1


In [32]:
y_test

,rating
1935,1
6494,0
1720,0
9120,0
360,0
...,...
1195,1
11877,1
5421,1
3855,1


### BOW and TFIDF

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [70]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [71]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
y_pred_bow=nb_model_bow.predict(X_test_bow)
y_pred_tfidf=nb_model_tfidf.predict(X_test_tfidf)

In [72]:
print("BOW Accuracy:",accuracy_score(y_test,y_pred_bow))

BOW Accuracy: 0.5775


In [73]:
print("BOW Confusion Matrix:\n",confusion_matrix(y_test,y_pred_bow))

BOW Confusion Matrix:
 [[499 304]
 [710 887]]


In [74]:
print("BOW Classification Report:\n",classification_report(y_test,y_pred_bow))

BOW Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.62      0.50       803
           1       0.74      0.56      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400



In [75]:
print("TFIDF Accuracy:",accuracy_score(y_test,y_pred_tfidf))

TFIDF Accuracy: 0.57875


In [76]:
print("TFIDF Confusion Matrix:\n",confusion_matrix(y_test,y_pred_tfidf))

TFIDF Confusion Matrix:
 [[482 321]
 [690 907]]


In [78]:
print("TFIDF Classification Report:\n",classification_report(y_test,y_pred_tfidf))

TFIDF Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.60      0.49       803
           1       0.74      0.57      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.57      0.58      0.57      2400
weighted avg       0.63      0.58      0.59      2400



### Word2Vec

In [33]:
!pip install gensim

In [34]:
from gensim.models import Word2Vec

In [53]:
w2v_model=Word2Vec(sentences=X_train,vector_size=200,window=5,min_count=1,workers=4)

In [54]:
w2v_model.wv

In [55]:
## AvgWord2Vec
import numpy as np
def avg_word2vec(sentence,model):
  vectors=[model.wv[word] for word in sentence if word in model.wv]
  if vectors:
    return np.mean(vectors,axis=0)
  else:
    return np.zeros(model.vector_size)

In [56]:
X_train_vectors=np.array([avg_word2vec(sentence,w2v_model) for sentence in X_train])
X_test_vectors=np.array([avg_word2vec(sentence,w2v_model) for sentence in X_test])

In [57]:
X_train_vectors.shape

(9600, 200)

In [58]:
## Applying the ML Algorithm on Word2Vec (RandomForest)
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier().fit(X_train_vectors,y_train)

In [59]:
rf_model

RandomForestClassifier()

In [60]:
rf_model.score(X_test_vectors,y_test)

0.6625

In [61]:
y_pred_w2v=rf_model.predict(X_test_vectors)

In [62]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Word2Vec Accuracy:",accuracy_score(y_test,y_pred_w2v))

Word2Vec Accuracy: 0.6625


In [63]:
print("Word2Vec Classification Report:\n",classification_report(y_test,y_pred_w2v))

Word2Vec Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.13      0.20       803
           1       0.68      0.93      0.79      1597

    accuracy                           0.66      2400
   macro avg       0.58      0.53      0.50      2400
weighted avg       0.61      0.66      0.59      2400



In [64]:
print("Word2Vec Confusion Matrix:\n",confusion_matrix(y_test,y_pred_w2v))

Word2Vec Confusion Matrix:
 [[ 104  699]
 [ 111 1486]]
